# --- Day 18: Like a Rogue ---

As you enter this room, you hear a loud click! Some of the tiles in the floor here seem to be pressure plates for traps, and the trap you just triggered has run out of... whatever it tried to do to you. You doubt you'll be so lucky next time.

Upon closer examination, the traps and safe tiles in this room seem to follow a pattern. The tiles are arranged into rows that are all the same width; you take note of the safe tiles (.) and traps (^) in the first row (your puzzle input).

The type of tile (trapped or safe) in each row is based on the types of the tiles in the same position, and to either side of that position, in the previous row. (If either side is off either end of the row, it counts as "safe" because there isn't a trap embedded in the wall.)

For example, suppose you know the first row (with tiles marked by letters) and want to determine the next row (with tiles marked by numbers):

```
ABCDE
12345
```

The type of tile 2 is based on the types of tiles A, B, and C; the type of tile 5 is based on tiles D, E, and an imaginary "safe" tile. Let's call these three tiles from the previous row the left, center, and right tiles, respectively. Then, a new tile is a trap only in one of the following situations:

- Its left and center tiles are traps, but its right tile is not.
- Its center and right tiles are traps, but its left tile is not.
- Only its left tile is a trap.
- Only its right tile is a trap.

In any other situation, the new tile is safe.

Then, starting with the row ..^^., you can determine the next row by applying those rules to each new tile:

- The leftmost character on the next row considers the left (nonexistent, so we assume "safe"), center (the first ., which means "safe"), and right (the second ., also "safe") tiles on the previous row. Because all of the trap rules require a trap in at least one of the previous three tiles, the first tile on this new row is also safe, ..
- The second character on the next row considers its left (.), center (.), and right (^) tiles from the previous row. This matches the fourth rule: only the right tile is a trap. Therefore, the next tile in this new row is a trap, ^.
- The third character considers .^^, which matches the second trap rule: its center and right tiles are traps, but its left tile is not. Therefore, this tile is also a trap, ^.
The last two characters in this new row match the first and third rules, respectively, and so they are both also traps, ^.

After these steps, we now know the next row of tiles in the room: .^^^^. Then, we continue on to the next row, using the same rules, and get ^^..^. After determining two new rows, our map looks like this:

```
..^^.
.^^^^
^^..^
```

Here's a larger example with ten tiles per row and ten rows:

```
.^^.^.^^^^
^^^...^..^
^.^^.^.^^.
..^^...^^^
.^^^^.^^.^
^^..^.^^..
^^^^..^^^.
^..^^^^.^^
.^^^..^.^^
^^.^^^..^^
```

In ten rows, this larger example has 38 safe tiles.

**Starting with the map in your puzzle input, in a total of 40 rows (including the starting row), how many safe tiles are there?**

---

Starting with the inputs:

In [85]:
import numpy as np

with open(f'inputs/18.txt') as f:
    data = f.read().strip()
puzzle_row = list(data)
print(len(puzzle_row), puzzle_row.count(TRAP))
puzzle_row[:15]

100 51


['.', '.', '.', '.', '.', '.', '^', '.', '^', '^', '.', '.', '.', '.', '.']

In [64]:
test_row = list("..^^.")
test_row

['.', '.', '^', '^', '.']

first up, a function to determine if a tile is a trap or not:

In [36]:
TRAP = "^"
SAFE = "."

def is_trap(l,c,r):
    """takes in the 3 relevant tiles and returns true if trap"""
    
    if l == c == TRAP and r == SAFE: return True
    if c == r == TRAP and l == SAFE: return True
    if l == TRAP and c == r == SAFE: return True
    if r == TRAP and l == c == SAFE: return True
    
    return False

is_trap("^", "^", "."), is_trap(".", ".", ".")

(True, False)

To keep it simple, I've first made a function which takes in a row and returns the next row: (this could just return the entire grid but this makes it easier to test):

In [72]:
def make_row(row=test_row):
    """takes in a row and returns the next row"""
    width = len(row)
    next_row = [SAFE for _ in row]
    
    # first
    if is_trap(SAFE, row[0], row[1]):
        next_row[0] = TRAP
    
    # all the middle ones
    for i in range(1, width-1):
        l, c, r = row[i-1], row[i], row[i+1]
        if is_trap(l,c,r):
            next_row[i] = TRAP
    
    # last one
    if is_trap(row[-2], row[-1], SAFE):
        next_row[-1] = TRAP
    
    return next_row
        
print("1: ", test_row)
r = make_row()
print("2: ", r)
print("3: ", make_row(r))    

1:  ['.', '.', '^', '^', '.']
2:  ['.', '^', '^', '^', '^']
3:  ['^', '^', '.', '.', '^']


In [77]:
def make_grid(height=3, start_row=test_row):
    """returns a grid of  height y"""
    width = len(start_row)
    
    grid = []
    grid.append(start_row)
    
    for i in range(1, height):
        next_row = make_row(grid[i-1])
        grid.append(next_row)
        
    return grid

test_grid = make_grid()
test_grid

[['.', '.', '^', '^', '.'],
 ['.', '^', '^', '^', '^'],
 ['^', '^', '.', '.', '^']]

In [92]:
assert sum([x.count(SAFE) for x in make_grid(10, list(".^^.^.^^^^"))]) == 38

In [95]:
%time sum([x.count(SAFE) for x in make_grid(40, puzzle_row)])

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.07 ms


1963

`1963` is the answer to my puzzle input. Onwards to day 2:

# --- Part Two ---

How many safe tiles are there in a total of 400000 rows?

In [94]:
%time sum([x.count(SAFE) for x in make_grid(400000, puzzle_row)])

CPU times: user 35.1 s, sys: 148 ms, total: 35.2 s
Wall time: 35.3 s


20009568

`20009568` is the answer to my puzzle input. Luckily my part 1 solution was fast enough to compute part 2 in just 30 seconds. I could make it faster by only keeping the last row in memory.. so lets try that:

In [101]:
def solve_two(height=400000, row=puzzle_row):
    safe_count = row.count(SAFE)
    
    for i in range(1, height):
        row = make_row(row)
        safe_count += row.count(SAFE)
    
    return safe_count
    
%time solve_two()

CPU times: user 29.3 s, sys: 8 ms, total: 29.4 s
Wall time: 29.4 s


20009568

So its pretty much the same speed, implying its the computation which is taking up all the time, not storing all the rows. For a bigger number of rows discarding the previous rows makes sense, but in this case it didn't. 

Using numpy would probably speed this up too.